# Business List Quick Formatting

In [1]:
import os
import pandas as pd
import numpy as np
import re

### Read Files and Concatenate
The directory should contain all of the files that will be used for the business list.   
After putting the directory path in as the function's parameter, this will read all of the files and create a single dataframe to work with.

In [2]:
def open_files(path):
    folder = path
    df_list = []
    for file in os.listdir(folder):
        if file.endswith('.csv'):
            file_path=os.path.join(folder,file)
            df = pd.read_csv(file_path)
            df_list.append(df)
    final = pd.concat(df_list, ignore_index=True)
    return final

In [6]:
df = open_files('/Users/josephmac/Downloads/Intermountain Health/')
#df = open_files('C:/Data/Intermountain Health/')

### Format Data to Fit Template

In [7]:
#Adjust Capitalization
df = df.apply(lambda col:col.str.title() if col.dtypes == 'object' else col)

In [9]:
#Get Formatted Address
cities = pd.read_csv('/Users/josephmac/Downloads/utah_cities.csv')
#cities = pd.read_csv('C:/Data/utah_cities.csv')
cities['Place'] = cities['Place'].str.split(' City').str[0]
cities['Place'] = cities['Place'].str.split(' Town').str[0]
cities['Place'] = cities['Place'].str.split(' Cdp').str[0]
cities_list = cities['Place'].to_list()
for city in cities['Place']:
    mask = df['Address'].str.contains(city, na=False)
    if mask.any():
        # Split only the rows that match
        df.loc[mask, 'street'] = df.loc[mask, 'Address'].str.split(city).str[0].str.strip()
        df.loc[mask, 'city'] = city  
        df['zip_candidates'] = df['Address'].str.findall(r'(\d{5})')
        df['zip'] = df['zip_candidates'].apply(lambda x: x[-1] if x else None)

In [11]:
#Insert NAICS Description
description = pd.read_csv('/Users/josephmac/Downloads/naics.csv')
#description = pd.read_csv('C:/Data/naics.csv')
description['Code'] = description['Code'].astype(int)
df = df.merge(description[['Code','Name']],left_on='NAICS',right_on='Code',how='left')
df

,Company Name,Patriot Participation,Address,County,City,Phone,Employees,NAICS,street,city,zip_candidates,zip,Code,Name
0,Sheriffs Dept,N,3075 N Main St Spanish Fork Ut 84660,Utah,Spanish Fork,(801) 379-8320,500-999,922120,3075 N Main St,Spanish Fork,[84660],84660,922120,Police Protection
1,Smokey'S Smokehouse,N,195 E 1600 N Spanish Fork Ut 846601013,Utah,Spanish Fork,(801) 894-1118,500-999,311111,195 E 1600 N,Spanish Fork,[84660],84660,311111,Dog and Cat Food Manufacturing
2,Costco Wholesale Corp,N,273 E 1000 N Spanish Fork Ut 84660,Utah,Spanish Fork,0,250-499,455211,273 E 1000 N,Spanish Fork,[84660],84660,455211,Warehouse Clubs and Supercenters
3,"Hydro Extrusion Usa, Llc",Y,1550 Hydro Way Spanish Fork Ut 846601360,Utah,Spanish Fork,(314) 997-2100,250-499,331318,1550 Hydro Way,Spanish Fork,[84660],84660,331318,"Other Aluminum Rolling, Drawing, and Extruding"
4,"M&T Enterprises, Llc",N,4604 W 5600 S Spanish Fork Ut 846604306,Utah,Spanish Fork,(435) 669-8546,250-499,561730,4604 W 5600 S,Spanish Fork,[84660],84660,561730,Landscaping Services
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3447,"Translation One, Llc",N,884 N 1040 W Mapleton Ut 846643226,Utah,Mapleton,(801) 836-8585,0,541930,884 N 1040 W,Mapleton,[84664],84664,541930,Translation and Interpretation Services
3448,"Woodco Industries, Inc.",N,2142 W Yarrow Dr Mapleton Ut 846644620,Utah,Mapleton,(801) 796-0890,0,524113,2142 W Yarrow Dr,Mapleton,[84664],84664,524113,Direct Life Insurance Carriers
3449,"Young Marketing, Llc",N,1441 S 800 W Mapleton Ut 846644213,Utah,Mapleton,(801) 888-8852,0,541611,1441 S 800 W,Mapleton,[84664],84664,541611,Administrative Management and General Manageme...
3450,"Zebra Cleaning Utah, Llc",N,175 W 1000 N Mapleton Ut 846643586,Utah,Mapleton,(801) 404-4611,0,561720,175 W 1000 N,Mapleton,[84664],84664,561720,Janitorial Services


### Export File

In [12]:
#df.to_csv('C:/Data/Business Lists/intermountain_health.csv')
df.to_csv('/Users/josephmac/Downloads/Intermountain Health Data.csv')